In [39]:
import time
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tf

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def conv_layer(prev_layer, K, F, S):
    width_prev_layer = int(prev_layer.get_shape()[3])
    Weight_mat = weight_variable([F, F, width_prev_layer, K])
    return tf.nn.conv2d(prev_layer, Weight_mat, strides=[1, S, S, 1], padding='SAME') + bias_variable([K])

def conv_pool_layer(prev_layer, K, F, S):
    """
    Convolutional Layer and followed by a pool 2x2 layer

    input:
    prev_layer: The previous layer
    K: Number of filters
    F: Spatial extent (width of each filter)
    S: The stride

    ouput:
    A layer with filters
    Let W be the width of prev. layer
    the output size is W/(2*S)
    """
    activation_layer = tf.nn.relu(conv_layer(prev_layer, K, F, S))
    return max_pool_2x2(activation_layer)

def Full_conect_layer(prev_layer, activation, K):
    """
    transform the previous layer to a "flat" matrix
    And then fully connect to the next layer

    input:
    prev_layer: The previous layer
    activation: ghd activation function of the nervous
    K: size of the output layer 

    ouput:
    a "flat" matrix with size K
    """
    sz_prev = int(reduce(lambda x,y: x*y, prev_layer.get_shape()[1:]))
    flat = tf.reshape(prev_layer, [-1, sz_prev])
    Weight = weight_variable([sz_prev, K])
    Bias = bias_variable([K])
    return activation(tf.matmul(flat, Weight) + Bias)

x_image = tf.reshape(x, [-1, 28, 28, 1])

cp1 = conv_pool_layer(x_image, 32, 5, 1)
cp2 = conv_pool_layer(cp1, 64, 5, 1)

fc1 = Full_conect_layer(cp2, tf.nn.relu, 1024)

keep_prob = tf.placeholder(tf.float32)
fc1_drop = tf.nn.dropout(fc1, keep_prob)

y_conv = Full_conect_layer(fc1_drop, tf.nn.softmax, 10)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [40]:
start_train_time = time.time()

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
for i in range(500):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
sess.close()

end_train_time = time.time()
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
print "Total Training Time:",(end_train_time - start_train_time)

step 0, training accuracy 0.14
step 100, training accuracy 0.86
step 200, training accuracy 0.92
step 300, training accuracy 0.84
step 400, training accuracy 1


FailedPreconditionError: Attempting to use uninitialized value Variable_15
	 [[Node: Variable_15/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_15"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_15)]]
Caused by op u'Variable_15/read', defined at:
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/p